In [ ]:
import numpy as np
import time
import pandas as pd
import glob
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf
import os
import time
import random
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_path = "./DATA/"

data_type = "ap"   # ap or lateral
rnd_seed = str(time.time())[-3:]
run_name_origin = data_type+"_"+"--"+rnd_seed


size_w = 500
size_h = 600
tr_ratio = 0.6
te_ratio = 0.2

learning_rate = 3e-5
epoch = 200

print(run_name_origin)

In [ ]:
def load_one_image(input_path):
    img = Image.open(input_path)
    img = img.resize([size_w,size_h])
    rslt = np.asarray(img)/255
    
    return rslt

def load_many_image(input_path):
    rslt = np.zeros([len(input_path),size_h,size_w,3])
    for i,img_path in enumerate(input_path):
        rslt[i] = load_one_image(img_path)
        
    return rslt

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
# normal =0, abn = 1
all_path_0 = glob.glob(data_path+"normal/"+data_type+"/*.jpg")
all_path_1_origin = glob.glob(data_path+"abnormal/"+data_type+"/*.jpg")

In [ ]:
random.seed(int(rnd_seed))
random.shuffle(all_path_0)
random.shuffle(all_path_1_origin)

In [ ]:
all_path_1_1 = all_path_1_origin[:208]
all_path_1_2 = all_path_1_origin[208:208*2]
all_path_1_3 = all_path_1_origin[208*2:208*3]
all_path_1_4 = all_path_1_origin[208*3:208*4]
all_path_1_5 = all_path_1_origin[208*4:]

In [ ]:



all_path_1 = all_path_1_1

run_name = run_name_origin+"_1"


# normal =0, abn = 1
path_tr_0 = all_path_0[:111]
path_tr_1 = all_path_1[:111]

path_vl_0 = all_path_0[int(len(all_path_0)*tr_ratio):int(len(all_path_0)*(1-te_ratio))]
path_vl_1 = all_path_1[int(len(all_path_0)*tr_ratio):int(len(all_path_1)*(1-te_ratio))]
path_vl_1 = path_vl_1[:len(path_vl_0)]

path_te_0 = all_path_0[-38:]
path_te_1 = all_path_1[-60:]

path_tr = np.concatenate([path_tr_0,path_tr_1])
path_vl = np.concatenate([path_vl_0,path_vl_1])
path_te = np.concatenate([path_te_0,path_te_1])

target_label_tr = ['normal']*len(path_tr_0) +['abnormal']*len(path_tr_1)
target_label_vl = ['normal']*len(path_vl_0) +['abnormal']*len(path_vl_1)
target_label_te = ['normal']*len(path_te_0) +['abnormal']*len(path_te_1)

target_tr = np.asarray(pd.get_dummies(target_label_tr))
target_vl = np.asarray(pd.get_dummies(target_label_vl))
target_te = np.asarray(pd.get_dummies(target_label_te))
# abnormal = [1,0]   normal = [0,1]


print( np.unique(target_label_tr,return_counts=True) )
print( np.unique(target_label_vl,return_counts=True) )
print( np.unique(target_label_te,return_counts=True) )
print(target_tr[:5])
print(target_label_tr[:5])


input_tr = load_many_image(path_tr)
input_vl = load_many_image(path_vl)
input_te = load_many_image(path_te)

datagen_tr = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = [0.9,1.1],
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='constant',
        cval = 0)
datagen_te = ImageDataGenerator()

gen_tr = datagen_tr.flow(x = input_tr,y=target_tr,batch_size=8)
gen_vl = datagen_te.flow(x = input_vl,y=target_vl,batch_size=8)
gen_te = datagen_te.flow(x = input_te,y=target_te,batch_size=8)

step_per_epoch = int( len(input_tr)/8 )


base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics=['accuracy'])


callback_tb = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name+"/".format(run_name), profile_batch = 100000000)
callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath = run_name+"_best.hdf5",monitor='val_accuracy',
                                                verbose=1, save_best_only=True,mode='max', save_freq='epoch')




model.fit(x = gen_tr, epochs=epoch, 
          validation_data=gen_vl, 
          steps_per_epoch = step_per_epoch,
                    max_queue_size=15,
                    workers=8,
          callbacks=[callback_tb,callback_mc])



model.load_weights(run_name+"_best.hdf5")

from tensorflow.keras import backend as K
import cv2

def CAM_multiclass(model,img_set,which_label=None):
    class_weights = model.layers[-1].get_weights()[0]
    final_conv = model.layers[-3]
    get_output = K.function([model.layers[0].input],[final_conv.output,model.layers[-1].output])

    cam_output = np.zeros(shape = [img_set.shape[0],img_set.shape[1],img_set.shape[2],3])

    for img_index in range(img_set.shape[0]):
        input_img = img_set[img_index:img_index+1,...]
        #input_img_rgb = np.concatenate([input_img, input_img, input_img], axis = 3)
        [conv_output,predictions] = get_output([input_img])
        conv_output = conv_output[0,...]
        pred_label = np.argmax(predictions)

        cam = np.zeros(shape = conv_output.shape[:-1])
        if which_label==None:
            for i, w in enumerate(class_weights[:, pred_label]): 
                cam += w * conv_output[...,i]
        if not (which_label==None):
            for i, w in enumerate(class_weights[:,  which_label[img_index] ]):  
                cam += w * conv_output[...,i]
        cam /= np.max(cam)

        cam_resize = cv2.resize(cam[:,:], (img_set.shape[2],img_set.shape[1]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam_resize), cv2.COLORMAP_JET)
        heatmap = heatmap[...,(2,1,0)]
        heatmap[np.where(cam_resize < 0.1)] = 0
        cam_output[img_index,:,:,:] = heatmap*0.3 + input_img[0,:,:,:]*255
        #cam_output[img_index,:,:,:] = heatmap*0.5 + input_img_rgb[0,:,:,:]
            
    return cam_output

pred_te = model.predict(input_te, batch_size=4)
pred_label_te = np.argmax(pred_te,axis=1).astype("str")
pred_label_te = np.where(pred_label_te=="1", "normal", pred_label_te)
pred_label_te = np.where(pred_label_te=="0", "abnormal", pred_label_te)

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, classification_report

auc_te = roc_auc_score(target_te[:,0],pred_te[:,0])

acc_te = accuracy_score(target_label_te, pred_label_te)
cf_mtx = confusion_matrix(target_label_te,pred_label_te)
pr_re_f1_report = classification_report(y_true = target_label_te, y_pred = pred_label_te)


result_report = pd.DataFrame(data = {"ID":[s.split("/")[-1] for s in path_te]
                                     ,"GT_Label" : target_label_te 
                                     ,"pred_Label" : pred_label_te
                                 ,"GT" : [str(s) for s in target_te ]
                                 ,"pred" : [str(s) for s in np.round(pred_te,3) ]
                                 ,"Correct?" : np.array(target_label_te)==np.array(pred_label_te) })
result_report = result_report[['ID',"GT_Label","pred_Label",'GT','pred','Correct?']]


result_report.to_csv("pred_rslt_TE"+run_name+".csv",index=False)

pd.set_option('display.width', 999999)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 99999)

with open("Report-"+run_name+".txt",'w') as text_file:
    print("",file=text_file)
    print("Overall accuracy on test set : ",acc_te,file=text_file)
    print("AUC on test set : ",auc_te,file=text_file)
    print("",file=text_file)
    print("Confusion Matrix (row=actual, col=predicted)",file=text_file)
    print("abnormal / normal",file=text_file)
#     print(class_name_list,file=text_file)
    print(cf_mtx,file=text_file)
    print("",file=text_file)
    print("Classification Report :",file=text_file)
    print(pr_re_f1_report,file=text_file)
    print("------------------RESULT REPORT--------------------",file=text_file)
    print(result_report,file=text_file)
pd.reset_option('display.width')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

cam_rslt_te = CAM_multiclass(model,input_te)



os.makedirs("./CAM/"+run_name,exist_ok=True)
for i in range(len(pred_label_te)):
    cv2.imwrite("./CAM/"+run_name+"/target"+str(target_label_te[i])+"pred"+str(pred_label_te[i])+"-ID_"+str(path_te[i].split("\\")[-1])
               ,cam_rslt_te[i])


In [ ]:



all_path_1 = all_path_1_2
run_name = run_name_origin+"_2"



# normal =0, abn = 1
path_tr_0 = all_path_0[:111]
path_tr_1 = all_path_1[:111]

path_vl_0 = all_path_0[int(len(all_path_0)*tr_ratio):int(len(all_path_0)*(1-te_ratio))]
path_vl_1 = all_path_1[int(len(all_path_0)*tr_ratio):int(len(all_path_1)*(1-te_ratio))]
path_vl_1 = path_vl_1[:len(path_vl_0)]

path_te_0 = all_path_0[-38:]
path_te_1 = all_path_1[-60:]

path_tr = np.concatenate([path_tr_0,path_tr_1])
path_vl = np.concatenate([path_vl_0,path_vl_1])
path_te = np.concatenate([path_te_0,path_te_1])

target_label_tr = ['normal']*len(path_tr_0) +['abnormal']*len(path_tr_1)
target_label_vl = ['normal']*len(path_vl_0) +['abnormal']*len(path_vl_1)
target_label_te = ['normal']*len(path_te_0) +['abnormal']*len(path_te_1)

target_tr = np.asarray(pd.get_dummies(target_label_tr))
target_vl = np.asarray(pd.get_dummies(target_label_vl))
target_te = np.asarray(pd.get_dummies(target_label_te))
# abnormal = [1,0]   normal = [0,1]


print( np.unique(target_label_tr,return_counts=True) )
print( np.unique(target_label_vl,return_counts=True) )
print( np.unique(target_label_te,return_counts=True) )
print(target_tr[:5])
print(target_label_tr[:5])


input_tr = load_many_image(path_tr)
input_vl = load_many_image(path_vl)
input_te = load_many_image(path_te)

datagen_tr = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = [0.9,1.1],
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='constant',
        cval = 0)
datagen_te = ImageDataGenerator()

gen_tr = datagen_tr.flow(x = input_tr,y=target_tr,batch_size=8)
gen_vl = datagen_te.flow(x = input_vl,y=target_vl,batch_size=8)
gen_te = datagen_te.flow(x = input_te,y=target_te,batch_size=8)

step_per_epoch = int( len(input_tr)/8 )


base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics=['accuracy'])


callback_tb = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name+"/".format(run_name), profile_batch = 100000000)
callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath = run_name+"_best.hdf5",monitor='val_accuracy',
                                                verbose=1, save_best_only=True,mode='max', save_freq='epoch')




model.fit(x = gen_tr, epochs=epoch, 
          validation_data=gen_vl, 
          steps_per_epoch = step_per_epoch,
                    max_queue_size=15,
                    workers=8,
          callbacks=[callback_tb,callback_mc])



model.load_weights(run_name+"_best.hdf5")

from tensorflow.keras import backend as K
import cv2

def CAM_multiclass(model,img_set,which_label=None):
    class_weights = model.layers[-1].get_weights()[0]
    final_conv = model.layers[-3]
    get_output = K.function([model.layers[0].input],[final_conv.output,model.layers[-1].output])

    cam_output = np.zeros(shape = [img_set.shape[0],img_set.shape[1],img_set.shape[2],3])

    for img_index in range(img_set.shape[0]):
        input_img = img_set[img_index:img_index+1,...]
        #input_img_rgb = np.concatenate([input_img, input_img, input_img], axis = 3)
        [conv_output,predictions] = get_output([input_img])
        conv_output = conv_output[0,...]
        pred_label = np.argmax(predictions)

        cam = np.zeros(shape = conv_output.shape[:-1])
        if which_label==None:
            for i, w in enumerate(class_weights[:, pred_label]): 
                cam += w * conv_output[...,i]
        if not (which_label==None):
            for i, w in enumerate(class_weights[:,  which_label[img_index] ]): 
                cam += w * conv_output[...,i]
        cam /= np.max(cam)

        cam_resize = cv2.resize(cam[:,:], (img_set.shape[2],img_set.shape[1]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam_resize), cv2.COLORMAP_JET)
        heatmap = heatmap[...,(2,1,0)]
        heatmap[np.where(cam_resize < 0.1)] = 0
        cam_output[img_index,:,:,:] = heatmap*0.3 + input_img[0,:,:,:]*255
        #cam_output[img_index,:,:,:] = heatmap*0.5 + input_img_rgb[0,:,:,:]
            
    return cam_output

pred_te = model.predict(input_te, batch_size=4)
pred_label_te = np.argmax(pred_te,axis=1).astype("str")
pred_label_te = np.where(pred_label_te=="1", "normal", pred_label_te)
pred_label_te = np.where(pred_label_te=="0", "abnormal", pred_label_te)

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, classification_report

auc_te = roc_auc_score(target_te[:,0],pred_te[:,0])

acc_te = accuracy_score(target_label_te, pred_label_te)
cf_mtx = confusion_matrix(target_label_te,pred_label_te)
pr_re_f1_report = classification_report(y_true = target_label_te, y_pred = pred_label_te)


result_report = pd.DataFrame(data = {"ID":[s.split("/")[-1] for s in path_te]
                                     ,"GT_Label" : target_label_te 
                                     ,"pred_Label" : pred_label_te
                                 ,"GT" : [str(s) for s in target_te ]
                                 ,"pred" : [str(s) for s in np.round(pred_te,3) ]
                                 ,"Correct?" : np.array(target_label_te)==np.array(pred_label_te) })
result_report = result_report[['ID',"GT_Label","pred_Label",'GT','pred','Correct?']]


result_report.to_csv("pred_rslt_TE"+run_name+".csv",index=False)

pd.set_option('display.width', 999999)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 99999)

with open("Report-"+run_name+".txt",'w') as text_file:
    print("",file=text_file)
    print("Overall accuracy on test set : ",acc_te,file=text_file)
    print("AUC on test set : ",auc_te,file=text_file)
    print("",file=text_file)
    print("Confusion Matrix (row=actual, col=predicted)",file=text_file)
    print("abnormal / normal",file=text_file)
#     print(class_name_list,file=text_file)
    print(cf_mtx,file=text_file)
    print("",file=text_file)
    print("Classification Report :",file=text_file)
    print(pr_re_f1_report,file=text_file)
    print("------------------RESULT REPORT--------------------",file=text_file)
    print(result_report,file=text_file)
pd.reset_option('display.width')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

cam_rslt_te = CAM_multiclass(model,input_te)



os.makedirs("./CAM/"+run_name,exist_ok=True)
for i in range(len(pred_label_te)):
    cv2.imwrite("./CAM/"+run_name+"/target"+str(target_label_te[i])+"pred"+str(pred_label_te[i])+"-ID_"+str(path_te[i].split("\\")[-1])
               ,cam_rslt_te[i])


In [ ]:



all_path_1 = all_path_1_3
run_name = run_name_origin+"_3"



# normal =0, abn = 1
path_tr_0 = all_path_0[:111]
path_tr_1 = all_path_1[:111]

path_vl_0 = all_path_0[int(len(all_path_0)*tr_ratio):int(len(all_path_0)*(1-te_ratio))]
path_vl_1 = all_path_1[int(len(all_path_0)*tr_ratio):int(len(all_path_1)*(1-te_ratio))]
path_vl_1 = path_vl_1[:len(path_vl_0)]

path_te_0 = all_path_0[-38:]
path_te_1 = all_path_1[-60:]

path_tr = np.concatenate([path_tr_0,path_tr_1])
path_vl = np.concatenate([path_vl_0,path_vl_1])
path_te = np.concatenate([path_te_0,path_te_1])

target_label_tr = ['normal']*len(path_tr_0) +['abnormal']*len(path_tr_1)
target_label_vl = ['normal']*len(path_vl_0) +['abnormal']*len(path_vl_1)
target_label_te = ['normal']*len(path_te_0) +['abnormal']*len(path_te_1)

target_tr = np.asarray(pd.get_dummies(target_label_tr))
target_vl = np.asarray(pd.get_dummies(target_label_vl))
target_te = np.asarray(pd.get_dummies(target_label_te))
# abnormal = [1,0]   normal = [0,1]


print( np.unique(target_label_tr,return_counts=True) )
print( np.unique(target_label_vl,return_counts=True) )
print( np.unique(target_label_te,return_counts=True) )
print(target_tr[:5])
print(target_label_tr[:5])


input_tr = load_many_image(path_tr)
input_vl = load_many_image(path_vl)
input_te = load_many_image(path_te)

datagen_tr = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = [0.9,1.1],
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='constant',
        cval = 0)
datagen_te = ImageDataGenerator()

gen_tr = datagen_tr.flow(x = input_tr,y=target_tr,batch_size=8)
gen_vl = datagen_te.flow(x = input_vl,y=target_vl,batch_size=8)
gen_te = datagen_te.flow(x = input_te,y=target_te,batch_size=8)

step_per_epoch = int( len(input_tr)/8 )


base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics=['accuracy'])


callback_tb = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name+"/".format(run_name), profile_batch = 100000000)
callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath = run_name+"_best.hdf5",monitor='val_accuracy',
                                                verbose=1, save_best_only=True,mode='max', save_freq='epoch')




model.fit(x = gen_tr, epochs=epoch, 
          validation_data=gen_vl, 
          steps_per_epoch = step_per_epoch,
                    max_queue_size=15,
                    workers=8,
          callbacks=[callback_tb,callback_mc])



model.load_weights(run_name+"_best.hdf5")

from tensorflow.keras import backend as K
import cv2

def CAM_multiclass(model,img_set,which_label=None):
    class_weights = model.layers[-1].get_weights()[0]
    final_conv = model.layers[-3]
    get_output = K.function([model.layers[0].input],[final_conv.output,model.layers[-1].output])

    cam_output = np.zeros(shape = [img_set.shape[0],img_set.shape[1],img_set.shape[2],3])

    for img_index in range(img_set.shape[0]):
        input_img = img_set[img_index:img_index+1,...]
        #input_img_rgb = np.concatenate([input_img, input_img, input_img], axis = 3)
        [conv_output,predictions] = get_output([input_img])
        conv_output = conv_output[0,...]
        pred_label = np.argmax(predictions)

        cam = np.zeros(shape = conv_output.shape[:-1])
        if which_label==None:
            for i, w in enumerate(class_weights[:, pred_label]): 
                cam += w * conv_output[...,i]
        if not (which_label==None):
            for i, w in enumerate(class_weights[:,  which_label[img_index] ]):  
                cam += w * conv_output[...,i]
        cam /= np.max(cam)

        cam_resize = cv2.resize(cam[:,:], (img_set.shape[2],img_set.shape[1]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam_resize), cv2.COLORMAP_JET)
        heatmap = heatmap[...,(2,1,0)]
        heatmap[np.where(cam_resize < 0.1)] = 0
        cam_output[img_index,:,:,:] = heatmap*0.3 + input_img[0,:,:,:]*255
        #cam_output[img_index,:,:,:] = heatmap*0.5 + input_img_rgb[0,:,:,:]
            
    return cam_output

pred_te = model.predict(input_te, batch_size=4)
pred_label_te = np.argmax(pred_te,axis=1).astype("str")
pred_label_te = np.where(pred_label_te=="1", "normal", pred_label_te)
pred_label_te = np.where(pred_label_te=="0", "abnormal", pred_label_te)

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, classification_report

auc_te = roc_auc_score(target_te[:,0],pred_te[:,0])

acc_te = accuracy_score(target_label_te, pred_label_te)
cf_mtx = confusion_matrix(target_label_te,pred_label_te)
pr_re_f1_report = classification_report(y_true = target_label_te, y_pred = pred_label_te)


result_report = pd.DataFrame(data = {"ID":[s.split("/")[-1] for s in path_te]
                                     ,"GT_Label" : target_label_te 
                                     ,"pred_Label" : pred_label_te
                                 ,"GT" : [str(s) for s in target_te ]
                                 ,"pred" : [str(s) for s in np.round(pred_te,3) ]
                                 ,"Correct?" : np.array(target_label_te)==np.array(pred_label_te) })
result_report = result_report[['ID',"GT_Label","pred_Label",'GT','pred','Correct?']]


result_report.to_csv("pred_rslt_TE"+run_name+".csv",index=False)

pd.set_option('display.width', 999999)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 99999)

with open("Report-"+run_name+".txt",'w') as text_file:
    print("",file=text_file)
    print("Overall accuracy on test set : ",acc_te,file=text_file)
    print("AUC on test set : ",auc_te,file=text_file)
    print("",file=text_file)
    print("Confusion Matrix (row=actual, col=predicted)",file=text_file)
    print("abnormal / normal",file=text_file)
#     print(class_name_list,file=text_file)
    print(cf_mtx,file=text_file)
    print("",file=text_file)
    print("Classification Report :",file=text_file)
    print(pr_re_f1_report,file=text_file)
    print("------------------RESULT REPORT--------------------",file=text_file)
    print(result_report,file=text_file)
pd.reset_option('display.width')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

cam_rslt_te = CAM_multiclass(model,input_te)



os.makedirs("./CAM/"+run_name,exist_ok=True)
for i in range(len(pred_label_te)):
    cv2.imwrite("./CAM/"+run_name+"/target"+str(target_label_te[i])+"pred"+str(pred_label_te[i])+"-ID_"+str(path_te[i].split("\\")[-1])
               ,cam_rslt_te[i])


In [ ]:



all_path_1 = all_path_1_4

run_name = run_name_origin+"_4"


# normal =0, abn = 1
path_tr_0 = all_path_0[:111]
path_tr_1 = all_path_1[:111]

path_vl_0 = all_path_0[int(len(all_path_0)*tr_ratio):int(len(all_path_0)*(1-te_ratio))]
path_vl_1 = all_path_1[int(len(all_path_0)*tr_ratio):int(len(all_path_1)*(1-te_ratio))]
path_vl_1 = path_vl_1[:len(path_vl_0)]

path_te_0 = all_path_0[-38:]
path_te_1 = all_path_1[-60:]

path_tr = np.concatenate([path_tr_0,path_tr_1])
path_vl = np.concatenate([path_vl_0,path_vl_1])
path_te = np.concatenate([path_te_0,path_te_1])

target_label_tr = ['normal']*len(path_tr_0) +['abnormal']*len(path_tr_1)
target_label_vl = ['normal']*len(path_vl_0) +['abnormal']*len(path_vl_1)
target_label_te = ['normal']*len(path_te_0) +['abnormal']*len(path_te_1)

target_tr = np.asarray(pd.get_dummies(target_label_tr))
target_vl = np.asarray(pd.get_dummies(target_label_vl))
target_te = np.asarray(pd.get_dummies(target_label_te))
# abnormal = [1,0]   normal = [0,1]


print( np.unique(target_label_tr,return_counts=True) )
print( np.unique(target_label_vl,return_counts=True) )
print( np.unique(target_label_te,return_counts=True) )
print(target_tr[:5])
print(target_label_tr[:5])


input_tr = load_many_image(path_tr)
input_vl = load_many_image(path_vl)
input_te = load_many_image(path_te)

datagen_tr = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = [0.9,1.1],
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='constant',
        cval = 0)
datagen_te = ImageDataGenerator()

gen_tr = datagen_tr.flow(x = input_tr,y=target_tr,batch_size=8)
gen_vl = datagen_te.flow(x = input_vl,y=target_vl,batch_size=8)
gen_te = datagen_te.flow(x = input_te,y=target_te,batch_size=8)

step_per_epoch = int( len(input_tr)/8 )


base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics=['accuracy'])


callback_tb = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name+"/".format(run_name), profile_batch = 100000000)
callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath = run_name+"_best.hdf5",monitor='val_accuracy',
                                                verbose=1, save_best_only=True,mode='max', save_freq='epoch')




model.fit(x = gen_tr, epochs=epoch, 
          validation_data=gen_vl, 
          steps_per_epoch = step_per_epoch,
                    max_queue_size=15,
                    workers=8,
          callbacks=[callback_tb,callback_mc])



model.load_weights(run_name+"_best.hdf5")

from tensorflow.keras import backend as K
import cv2

def CAM_multiclass(model,img_set,which_label=None):
    class_weights = model.layers[-1].get_weights()[0]
    final_conv = model.layers[-3]
    get_output = K.function([model.layers[0].input],[final_conv.output,model.layers[-1].output])

    cam_output = np.zeros(shape = [img_set.shape[0],img_set.shape[1],img_set.shape[2],3])

    for img_index in range(img_set.shape[0]):
        input_img = img_set[img_index:img_index+1,...]
        #input_img_rgb = np.concatenate([input_img, input_img, input_img], axis = 3)
        [conv_output,predictions] = get_output([input_img])
        conv_output = conv_output[0,...]
        pred_label = np.argmax(predictions)

        cam = np.zeros(shape = conv_output.shape[:-1])
        if which_label==None:
            for i, w in enumerate(class_weights[:, pred_label]): 
                cam += w * conv_output[...,i]
        if not (which_label==None):
            for i, w in enumerate(class_weights[:,  which_label[img_index] ]):  
                cam += w * conv_output[...,i]
        cam /= np.max(cam)

        cam_resize = cv2.resize(cam[:,:], (img_set.shape[2],img_set.shape[1]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam_resize), cv2.COLORMAP_JET)
        heatmap = heatmap[...,(2,1,0)]
        heatmap[np.where(cam_resize < 0.1)] = 0
        cam_output[img_index,:,:,:] = heatmap*0.3 + input_img[0,:,:,:]*255
        #cam_output[img_index,:,:,:] = heatmap*0.5 + input_img_rgb[0,:,:,:]
            
    return cam_output

pred_te = model.predict(input_te, batch_size=4)
pred_label_te = np.argmax(pred_te,axis=1).astype("str")
pred_label_te = np.where(pred_label_te=="1", "normal", pred_label_te)
pred_label_te = np.where(pred_label_te=="0", "abnormal", pred_label_te)

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, classification_report

auc_te = roc_auc_score(target_te[:,0],pred_te[:,0])

acc_te = accuracy_score(target_label_te, pred_label_te)
cf_mtx = confusion_matrix(target_label_te,pred_label_te)
pr_re_f1_report = classification_report(y_true = target_label_te, y_pred = pred_label_te)


result_report = pd.DataFrame(data = {"ID":[s.split("/")[-1] for s in path_te]
                                     ,"GT_Label" : target_label_te 
                                     ,"pred_Label" : pred_label_te
                                 ,"GT" : [str(s) for s in target_te ]
                                 ,"pred" : [str(s) for s in np.round(pred_te,3) ]
                                 ,"Correct?" : np.array(target_label_te)==np.array(pred_label_te) })
result_report = result_report[['ID',"GT_Label","pred_Label",'GT','pred','Correct?']]


result_report.to_csv("pred_rslt_TE"+run_name+".csv",index=False)

pd.set_option('display.width', 999999)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 99999)

with open("Report-"+run_name+".txt",'w') as text_file:
    print("",file=text_file)
    print("Overall accuracy on test set : ",acc_te,file=text_file)
    print("AUC on test set : ",auc_te,file=text_file)
    print("",file=text_file)
    print("Confusion Matrix (row=actual, col=predicted)",file=text_file)
    print("abnormal / normal",file=text_file)
#     print(class_name_list,file=text_file)
    print(cf_mtx,file=text_file)
    print("",file=text_file)
    print("Classification Report :",file=text_file)
    print(pr_re_f1_report,file=text_file)
    print("------------------RESULT REPORT--------------------",file=text_file)
    print(result_report,file=text_file)
pd.reset_option('display.width')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

cam_rslt_te = CAM_multiclass(model,input_te)



os.makedirs("./CAM/"+run_name,exist_ok=True)
for i in range(len(pred_label_te)):
    cv2.imwrite("./CAM/"+run_name+"/target"+str(target_label_te[i])+"pred"+str(pred_label_te[i])+"-ID_"+str(path_te[i].split("\\")[-1])
               ,cam_rslt_te[i])


In [ ]:



all_path_1 = all_path_1_5
run_name = run_name_origin+"_5"



# normal =0, abn = 1
path_tr_0 = all_path_0[:111]
path_tr_1 = all_path_1[:111]

path_vl_0 = all_path_0[int(len(all_path_0)*tr_ratio):int(len(all_path_0)*(1-te_ratio))]
path_vl_1 = all_path_1[int(len(all_path_0)*tr_ratio):int(len(all_path_1)*(1-te_ratio))]
path_vl_1 = path_vl_1[:len(path_vl_0)]

path_te_0 = all_path_0[-38:]
path_te_1 = all_path_1[-60:]

path_tr = np.concatenate([path_tr_0,path_tr_1])
path_vl = np.concatenate([path_vl_0,path_vl_1])
path_te = np.concatenate([path_te_0,path_te_1])

target_label_tr = ['normal']*len(path_tr_0) +['abnormal']*len(path_tr_1)
target_label_vl = ['normal']*len(path_vl_0) +['abnormal']*len(path_vl_1)
target_label_te = ['normal']*len(path_te_0) +['abnormal']*len(path_te_1)

target_tr = np.asarray(pd.get_dummies(target_label_tr))
target_vl = np.asarray(pd.get_dummies(target_label_vl))
target_te = np.asarray(pd.get_dummies(target_label_te))
# abnormal = [1,0]   normal = [0,1]


print( np.unique(target_label_tr,return_counts=True) )
print( np.unique(target_label_vl,return_counts=True) )
print( np.unique(target_label_te,return_counts=True) )
print(target_tr[:5])
print(target_label_tr[:5])


input_tr = load_many_image(path_tr)
input_vl = load_many_image(path_vl)
input_te = load_many_image(path_te)

datagen_tr = ImageDataGenerator(
        rescale=1/255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        brightness_range = [0.9,1.1],
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='constant',
        cval = 0)
datagen_te = ImageDataGenerator()

gen_tr = datagen_tr.flow(x = input_tr,y=target_tr,batch_size=8)
gen_vl = datagen_te.flow(x = input_vl,y=target_vl,batch_size=8)
gen_te = datagen_te.flow(x = input_te,y=target_te,batch_size=8)

step_per_epoch = int( len(input_tr)/8 )


base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),metrics=['accuracy'])


callback_tb = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name+"/".format(run_name), profile_batch = 100000000)
callback_mc = tf.keras.callbacks.ModelCheckpoint(filepath = run_name+"_best.hdf5",monitor='val_accuracy',
                                                verbose=1, save_best_only=True,mode='max', save_freq='epoch')




model.fit(x = gen_tr, epochs=epoch, 
          validation_data=gen_vl, 
          steps_per_epoch = step_per_epoch,
                    max_queue_size=15,
                    workers=8,
          callbacks=[callback_tb,callback_mc])



model.load_weights(run_name+"_best.hdf5")

from tensorflow.keras import backend as K
import cv2

def CAM_multiclass(model,img_set,which_label=None):
    class_weights = model.layers[-1].get_weights()[0]
    final_conv = model.layers[-3]
    get_output = K.function([model.layers[0].input],[final_conv.output,model.layers[-1].output])

    cam_output = np.zeros(shape = [img_set.shape[0],img_set.shape[1],img_set.shape[2],3])

    for img_index in range(img_set.shape[0]):
        input_img = img_set[img_index:img_index+1,...]
        #input_img_rgb = np.concatenate([input_img, input_img, input_img], axis = 3)
        [conv_output,predictions] = get_output([input_img])
        conv_output = conv_output[0,...]
        pred_label = np.argmax(predictions)

        cam = np.zeros(shape = conv_output.shape[:-1])
        if which_label==None:
            for i, w in enumerate(class_weights[:, pred_label]): 
                cam += w * conv_output[...,i]
        if not (which_label==None):
            for i, w in enumerate(class_weights[:,  which_label[img_index] ]): 
                cam += w * conv_output[...,i]
        cam /= np.max(cam)

        cam_resize = cv2.resize(cam[:,:], (img_set.shape[2],img_set.shape[1]))
        heatmap = cv2.applyColorMap(np.uint8(255*cam_resize), cv2.COLORMAP_JET)
        heatmap = heatmap[...,(2,1,0)]
        heatmap[np.where(cam_resize < 0.1)] = 0
        cam_output[img_index,:,:,:] = heatmap*0.3 + input_img[0,:,:,:]*255
        #cam_output[img_index,:,:,:] = heatmap*0.5 + input_img_rgb[0,:,:,:]
            
    return cam_output

pred_te = model.predict(input_te, batch_size=4)
pred_label_te = np.argmax(pred_te,axis=1).astype("str")
pred_label_te = np.where(pred_label_te=="1", "normal", pred_label_te)
pred_label_te = np.where(pred_label_te=="0", "abnormal", pred_label_te)

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, classification_report

auc_te = roc_auc_score(target_te[:,0],pred_te[:,0])

acc_te = accuracy_score(target_label_te, pred_label_te)
cf_mtx = confusion_matrix(target_label_te,pred_label_te)
pr_re_f1_report = classification_report(y_true = target_label_te, y_pred = pred_label_te)


result_report = pd.DataFrame(data = {"ID":[s.split("/")[-1] for s in path_te]
                                     ,"GT_Label" : target_label_te 
                                     ,"pred_Label" : pred_label_te
                                 ,"GT" : [str(s) for s in target_te ]
                                 ,"pred" : [str(s) for s in np.round(pred_te,3) ]
                                 ,"Correct?" : np.array(target_label_te)==np.array(pred_label_te) })
result_report = result_report[['ID',"GT_Label","pred_Label",'GT','pred','Correct?']]


result_report.to_csv("pred_rslt_TE"+run_name+".csv",index=False)

pd.set_option('display.width', 999999)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 99999)

with open("Report-"+run_name+".txt",'w') as text_file:
    print("",file=text_file)
    print("Overall accuracy on test set : ",acc_te,file=text_file)
    print("AUC on test set : ",auc_te,file=text_file)
    print("",file=text_file)
    print("Confusion Matrix (row=actual, col=predicted)",file=text_file)
    print("abnormal / normal",file=text_file)
#     print(class_name_list,file=text_file)
    print(cf_mtx,file=text_file)
    print("",file=text_file)
    print("Classification Report :",file=text_file)
    print(pr_re_f1_report,file=text_file)
    print("------------------RESULT REPORT--------------------",file=text_file)
    print(result_report,file=text_file)
pd.reset_option('display.width')
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

cam_rslt_te = CAM_multiclass(model,input_te)



os.makedirs("./CAM/"+run_name,exist_ok=True)
for i in range(len(pred_label_te)):
    cv2.imwrite("./CAM/"+run_name+"/target"+str(target_label_te[i])+"pred"+str(pred_label_te[i])+"-ID_"+str(path_te[i].split("\\")[-1])
               ,cam_rslt_te[i])
